In [1]:
import numpy as np, glob, ujson
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2   

# next cell
%reload_ext autoreload

In [2]:
import tensorflow as tf
print(tf.__version__)
import wavenet_tf as tf_wn

1.15.0


In [3]:
files = sorted(glob.glob("./data/fma_small_25_8000/*.npz"))

In [4]:
result = np.load(files[0])

In [5]:
X = result['audios']

In [6]:
ds = tf.data.Dataset.from_tensor_slices(X[:, :101025].reshape(X.shape[0], -1, 1))
ds = ds.batch(1)

In [7]:
X.shape

(100, 200000)

In [8]:
sess = tf.Session()

In [9]:
iter = ds.make_initializable_iterator()

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.


In [10]:
next_el = iter.get_next()

In [11]:
sess.run(iter.initializer, feed_dict={})

In [12]:
sess.run(next_el).shape

(1, 101025, 1)

In [13]:
with open("./data/tf_wavenet_params.json", "r") as f:
    wavenet_params = ujson.load(f)

In [14]:
net = tf_wn.WaveNetModel(
        batch_size=1,
        dilations=[1, 2, 4, 8, 16, 32, 64, 128, 256, 512],
        filter_width=2,
        residual_channels=32,
        dilation_channels=32,
        skip_channels=512,
        quantization_channels=256,
        use_biases=True,
        scalar_input=False,
        initial_filter_width=32,
        histograms=False,
        global_condition_channels=None,
        global_condition_cardinality=None)

In [15]:
loss = net.loss(input_batch=next_el, global_condition_batch=None, l2_regularization_strength=None)

In [16]:
init = tf.global_variables_initializer()
sess.run(init)

In [17]:
net.receptive_field

1025

In [18]:
sess.run(loss)

5.5470614

In [31]:
coord = tf.train.Coordinator()
reader = tf_wn.AudioReader(
    './data/fma_small/100',
    coord,
    sample_rate=wavenet_params['sample_rate'],
    gc_enabled=None,
    receptive_field=tf_wn.WaveNetModel.calculate_receptive_field(wavenet_params["filter_width"],
                                                           wavenet_params["dilations"],
                                                           wavenet_params["scalar_input"],
                                                           wavenet_params["initial_filter_width"]),
    sample_size=100000,
    silence_threshold=None)
audio_batch = reader.dequeue(1)

In [ ]:
x = sess.run(audio_batch)